In [50]:
import pandas as pd
import os
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, ElasticNetCV

import numpy as np

os.chdir('/data/arpit.goel/20_TwoSigmaFinancial')

train=pd.read_pickle('03.ProcessedData/train_with_predicts_ids.pkl').sample(n=100000)
test_id=pd.read_pickle('03.ProcessedData/test_id_with_predicts_ids.pkl')
test_ts=pd.read_pickle('03.ProcessedData/test_ts_with_predicts_ids.pkl')

msk = np.random.rand(len(train)) < 0.6
ins = train[msk]
oos = train[~msk]


In [31]:
cols=['p1','technical_30', 'technical_20', 'fundamental_11']
cols=['technical_20']
cols=['p%d'%x for x in range(6)]+['technical_19', 'technical_20', 'technical_10']
cols=['technical_30', 'technical_20', 'fundamental_11', 'technical_19']
cols = [x for x in train.columns.tolist() if x not in ['y','timestamp','id']]

gb = LinearRegression()
gb.fit(ins[cols],ins['y'])
print gb.score(ins[cols],ins['y']), gb.score(oos[cols],oos['y'])
print gb.score(test_ts[cols],test_ts['y']), gb.score(test_id[cols],test_id['y'])


0.0110932148018 -0.00656224191736
-0.0186651960873 -0.00913028661932


In [18]:
cols = [x for x in train.columns.tolist() if x not in ['y','timestamp','id']]

gb = GradientBoostingRegressor(n_estimators=5, max_depth=3)
gb.fit(ins[cols],ins['y'])
print gb.score(ins[cols],ins['y']), gb.score(oos[cols],oos['y'])
print gb.score(test_ts[cols],test_ts['y']), gb.score(test_id[cols],test_id['y'])


0.0132830748496 -0.00112268929687
-0.00387136845652 -0.000918686926743


In [19]:
feature_importance=pd.DataFrame(zip(cols,list(gb.feature_importances_)),columns=['variable','importance']).sort_values(by=['importance'],ascending=False)
feature_importance['Cumm']=feature_importance['importance'].cumsum()
feature_importance

,variable,importance,Cumm
84,technical_19,0.194915,0.194915
76,technical_10,0.161821,0.356736
85,technical_20,0.093515,0.450251
98,technical_35,0.064291,0.514542
46,fundamental_42,0.057437,0.571979
6,fundamental_1,0.051727,0.623706
36,fundamental_32,0.043935,0.667641
57,fundamental_53,0.040617,0.708258
23,fundamental_19,0.039863,0.748121
86,technical_21,0.038012,0.786133


In [10]:
ins[cols+['y']].corr()['y'].abs().sort_values(ascending=False)

y                 1.000000
p1                0.032839
technical_19      0.032171
technical_40      0.030846
technical_36      0.029884
fundamental_21    0.028256
technical_35      0.028200
technical_12      0.027957
fundamental_42    0.026778
fundamental_58    0.025759
technical_30      0.025233
fundamental_40    0.025203
technical_27      0.023722
fundamental_54    0.023019
p0                0.022485
fundamental_0     0.021999
fundamental_59    0.021254
fundamental_5     0.019082
technical_14      0.018792
derived_4         0.018544
fundamental_52    0.018018
fundamental_45    0.017865
technical_7       0.017858
fundamental_19    0.017590
fundamental_38    0.017434
derived_2         0.017149
technical_20      0.017039
technical_34      0.016832
technical_6       0.016613
technical_43      0.016212
                    ...   
fundamental_16    0.003887
fundamental_27    0.003696
fundamental_25    0.003659
derived_1         0.003563
technical_28      0.003350
fundamental_6     0.003130
d

In [41]:
def get_corr(data):
    return data[cols+['y']].corr()['y'].abs().sort_values(ascending=False)

ins.groupby(['group']).apply(get_corr).unstack().T.to_csv('03.ProcessedData/correlation_by_group.csv')
get_corr(ins).to_csv('03.ProcessedData/correlation_overall.csv')

In [49]:
idtagger_cols=['technical_7','technical_40','fundamental_39','fundamental_8','fundamental_14','fundamental_5','fundamental_40','fundamental_21',\
    'fundamental_58','fundamental_51','fundamental_0','fundamental_22', 'fundamental_46','fundamental_13','fundamental_16','fundamental_57',\
    'fundamental_23','derived_3','fundamental_61','fundamental_10','fundamental_24','fundamental_9','fundamental_12','fundamental_15','fundamental_59']

segmented_model_vars=[['fundamental_61', 'fundamental_11', 'fundamental_28', 'fundamental_2', 'fundamental_16'],\
                      ['fundamental_47', 'fundamental_31', 'fundamental_37', 'fundamental_3', 'fundamental_15'],\
                      ['fundamental_12', 'fundamental_21', 'derived_3', 'fundamental_41', 'fundamental_18'],\
                      ['fundamental_27', 'fundamental_6', 'fundamental_62', 'fundamental_34', 'fundamental_53'],\
                      ['derived_2', 'fundamental_41', 'fundamental_7', 'fundamental_26', 'fundamental_42'],\
                      ['fundamental_44', 'fundamental_0', 'fundamental_7', 'fundamental_36', 'fundamental_19']]

['id']


{'derived_2',
 'derived_3',
 'fundamental_0',
 'fundamental_10',
 'fundamental_11',
 'fundamental_12',
 'fundamental_13',
 'fundamental_14',
 'fundamental_15',
 'fundamental_16',
 'fundamental_18',
 'fundamental_19',
 'fundamental_2',
 'fundamental_21',
 'fundamental_22',
 'fundamental_23',
 'fundamental_24',
 'fundamental_26',
 'fundamental_27',
 'fundamental_28',
 'fundamental_3',
 'fundamental_31',
 'fundamental_34',
 'fundamental_36',
 'fundamental_37',
 'fundamental_39',
 'fundamental_40',
 'fundamental_41',
 'fundamental_42',
 'fundamental_44',
 'fundamental_46',
 'fundamental_47',
 'fundamental_5',
 'fundamental_51',
 'fundamental_53',
 'fundamental_57',
 'fundamental_58',
 'fundamental_59',
 'fundamental_6',
 'fundamental_61',
 'fundamental_62',
 'fundamental_7',
 'fundamental_8',
 'fundamental_9',
 'id',
 'technical_40',
 'technical_7'}

In [57]:
from sklearn.tree import DecisionTreeRegressor

cols = [x for x in train.columns.tolist() if x not in ['y','timestamp','id']]

gb = DecisionTreeRegressor(max_depth=1)

r2 = []
for col in cols:
    gb.fit(ins[[col]],ins['y'])
    r2.append([col, gb.score(ins[[col]],ins['y']), gb.score(oos[[col]],oos['y']), gb.score(test_ts[[col]],test_ts['y']), gb.score(test_id[[col]],test_id['y'])])
    
r2 = pd.DataFrame(r2,columns=['columns','ins','oos','test_ts','test_id'])

print r2.sort_values(by='ins').tail()
print r2.sort_values(by='oos').tail()
print r2.sort_values(by='test_ts').tail()
print r2.sort_values(by='test_id').tail()


           columns       ins       oos   test_ts       test_id
98    technical_35  0.000748 -0.000193 -0.000134 -2.605897e-08
90    technical_27  0.000823  0.000062 -0.001199 -3.142195e-04
93    technical_30  0.000829  0.000435 -0.000806  4.355287e-05
85    technical_20  0.000890 -0.000612 -0.001815  5.266490e-05
57  fundamental_53  0.000953 -0.001812 -0.001563 -1.431737e-03
           columns       ins       oos   test_ts   test_id
15  fundamental_11  0.000376  0.000230 -0.000457 -0.000448
84    technical_19  0.000660  0.000304 -0.001026 -0.000316
0        derived_0  0.000117  0.000313 -0.000725 -0.000087
99    technical_36  0.000660  0.000368 -0.000999 -0.000338
93    technical_30  0.000829  0.000435 -0.000806  0.000044
           columns       ins       oos   test_ts   test_id
92    technical_29  0.000124 -0.000032 -0.000040 -0.000048
60  fundamental_56  0.000188 -0.000170 -0.000040 -0.000015
80    technical_14  0.000033 -0.000019 -0.000016  0.000044
49  fundamental_45  0.000092  0.

In [67]:
train['col']=train[segmented_model_vars[0]].apply(lambda x: np.argmax(x),axis=1)
train[['col']+segmented_model_vars[0]].head()

,col,fundamental_61,fundamental_11,fundamental_28,fundamental_2,fundamental_16
98830,fundamental_16,0.074057,-0.400091,-0.205936,-0.344620,0.280767
8520,fundamental_61,0.074057,-1.422595,-0.380236,-0.939883,-0.035957
66684,fundamental_16,0.074057,-0.328835,-0.214811,-0.289546,0.184127
31157,fundamental_28,0.074057,0.016603,0.117488,-0.043684,0.042357
35223,fundamental_2,-0.117230,0.094020,0.048721,0.240272,-0.236161


In [71]:
[x for y in segmented_model_vars for x in y ]

['fundamental_61',
 'fundamental_11',
 'fundamental_28',
 'fundamental_2',
 'fundamental_16',
 'fundamental_47',
 'fundamental_31',
 'fundamental_37',
 'fundamental_3',
 'fundamental_15',
 'fundamental_12',
 'fundamental_21',
 'derived_3',
 'fundamental_41',
 'fundamental_18',
 'fundamental_27',
 'fundamental_6',
 'fundamental_62',
 'fundamental_34',
 'fundamental_53',
 'derived_2',
 'fundamental_41',
 'fundamental_7',
 'fundamental_26',
 'fundamental_42',
 'fundamental_44',
 'fundamental_0',
 'fundamental_7',
 'fundamental_36',
 'fundamental_19']

In [66]:
train[segmented_model_vars[0]]==train[segmented_model_vars[0]].max(axis=1)

MemoryError: 

In [62]:
train['col'].rank()

98830    18980.5
8520     82233.5
66684    18980.5
31157    53805.5
35223    36207.5
38169    18980.5
93451    82233.5
7108     18980.5
69346    53805.5
21925    53805.5
21734    18980.5
81528    18980.5
75148    36207.5
93549    36207.5
65602    18980.5
24636    82233.5
12934    53805.5
14663     4345.5
50088    18980.5
15130     4345.5
20767    36207.5
48114     4345.5
4779     36207.5
46307    82233.5
74219    36207.5
80107    82233.5
31113    18980.5
81485    53805.5
98263    82233.5
32540    82233.5
          ...   
11818    53805.5
93915     4345.5
2261     82233.5
64673    53805.5
18428    53805.5
24184    82233.5
52445    53805.5
83199     4345.5
21962    53805.5
85769    82233.5
55718    82233.5
55368    53805.5
61306     4345.5
28961    53805.5
3780     36207.5
89445    53805.5
94358    82233.5
70662    82233.5
47632    18980.5
51583    53805.5
14391    18980.5
16003    53805.5
1668     18980.5
83151    53805.5
12028    18980.5
74570    18980.5
73805    53805.5
39200    82233